##Loading Packages and Path

In [1]:
# pip install prince

In [2]:
#Import

import pandas as pd
import numpy as np
import re
import gc
import sys
import math
from datetime import date


import matplotlib.pyplot as plt
from sklearn.cluster import KMeans as kmean
import random
from scipy.spatial import distance
import array

from sklearn.decomposition import PCA

from statistics import mean
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import axes3d
#%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt


#import prince

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Main Path

path_main = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Raw Data/'
path_main_emp = path_main+'Employment/'
path_main_edu = path_main + 'Education/'
path_weekly_array = path_main_emp+'Employment History/Weekly Array/'
path_byJob = path_main_emp + 'Employment History/By Jobs/'
path_summary_array = path_main_emp+ 'Summary Measures/'

In [5]:
emp_hist_final_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Data Prep/Bikalpa/Temp_df/emp_hist_final.csv'
emp_hist_final = pd.read_csv(emp_hist_final_path)
emp_hist_final.drop('Unnamed: 0', inplace=True, axis=1)


#Creating the nine statistics 

Nine statistics to calculate 


1.   Fraction of jobs lasting less than 1 quarter
2.   Fraction of jobs lasting between 1 to 4 quarters 
3.   Fraction of jobs lasting between 5 to 8 quarters 
4.   Fraction of jobs lasting more than 8 quarters 
5.   Fraction of unemployment spells lasting less than 1 quarter
6.   Fraction of unemployment spells lasting between 1 an 4 quarters 
7.   Fraction of unemployment spells lasting between 4 to 8 quarters
8.   Total quarters of unemployment as a fraction of total number of quarters on record 
9.   Total number of different jobs as a fraction of the total number of quarters on record 







##Functions

In [6]:
def intersection(lst1, lst2):
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

def Diff(li1, li2):
    return list(set(li1) - set(li2))


def only_this_interval(q_minus1, q , q_plus1):
  not_before = Diff(q, q_minus1)
  not_after = Diff(q, q_plus1)
  only_this_q = intersection(not_before, not_after)
  return only_this_q #len(only_this)

def unpack_trackers(x_list, quarter):
  tracker_list = []
  for i in quarter:
    begin = 12*i
    end = 12*(i+1)
    tracker_all= list(set(x_list[begin:end]))
    trackers = [y for y in tracker_all if y>0 and y <100] #>= 999 are also considered unemployed for our purposes
    tracker_list = list(set(tracker_list + trackers))
  return tracker_list


In [7]:
#Employment Stats 

def stat_emp(x_list, length):
  
  num_emp_less = []
  pre = [y for y in range(0,length)]
  current = [y for y in range(length,2*length) ]
  
  first_iteration = 1 

  for quarter in range(length, 167 - 2 * length ):  
    
    post = [y for y in range(max(current)+1, max(current)+1+length)]
    
    trackers_pre = unpack_trackers(x_list,  pre)
    trackers_current = unpack_trackers(x_list, current)
    trackers_post = unpack_trackers(x_list, post)
    
    #Job the first length not caputured 
    if  first_iteration == 1:
      num_emp_less = Diff(trackers_pre , trackers_current)
      first_iteration = 0 

    #if item in post already in num_emp_less, take it out
    num_emp_less = Diff(num_emp_less, trackers_post)

    if len(trackers_current) != 0: 
      a = only_this_interval( trackers_pre, trackers_current , trackers_post)
      if len(a) != 0: 
        for item in a:
          if item not in num_emp_less: num_emp_less.append(item)
          

    pre = [y for y in range(min(current)-(length-1),    min(current)+1) ] 
    current = [y for y in range(min(post)-(length-1),    min(post)+1) ] 

  return num_emp_less


def employment_stats(x, tracker_list):
  total_jobs = 0 
  total_jobs_pre1 =  list(set(x[tracker_list].tolist()))
  total_jobs_pre2 = [y for y in total_jobs_pre1 if y>0 and y <999]
  total_jobs = len(total_jobs_pre2)       #This way jobs lasting less than a month are mostly excluded 
  
  x_list =  x[tracker_list].tolist()

  stat_1_list = stat_emp(x_list, 1)
  stat_2_list = stat_emp(x_list, 4)
  stat_3_list = stat_emp(x_list, 8)


  if total_jobs!=0:
    if len(stat_1_list) <= len(stat_2_list) and len(stat_2_list) <= len(stat_3_list):
      stat_1 = len(stat_1_list) / total_jobs
      stat_2 = (len(stat_2_list) - len(stat_1_list))/total_jobs
      stat_3 = (len(stat_3_list) - len(stat_2_list)) / total_jobs
      stat_4 = 1 - (stat_1+stat_2+stat_3)

      x["stat1"] = stat_1
      x["stat2"] = stat_2
      x["stat3"] = stat_3
      x["stat4"] = stat_4
    else:
      print(x["ID"])
  
    
  x["stat9"] = total_jobs / 167                    
  return x 


In [8]:
def sum_zero_before_1000(li,x):
  a = li.index(x)
  print(a)
  for i in range(0,a):
    if li[i] == 0: a = a -1
  print(a)
  return a  

test = [0, 0, 0, 0, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1001, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 1002, 1002, 1002, 1002, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 1003, 1003, 3.0, 3.0, 3.0, 3.0, 1004, 1004, 1004, 1004, 1004, 1004, 1004, 1004, 1004, 1004, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 1005, 1005, 1005, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

print(sum_zero_before_1000(test,1000))

4
0
0


In [9]:
#Unemployment Stats 

def last_occurance_1000(li, x):
    for i in range(0,len(li)):
        if li[i] == x and li[i+1]!=x:
            return i
    return "Error"

def sum_zero_before_1000(li,x):
  a = li.index(x)
  for i in range(0,a):
    if li[i] == 0: a = a -1
  return a   #This thing will return zero only if all numbers before x are zero



  
def impute_unemp_tracker(x_list):
  is_len_minus1 = 0 
  counter = 1000
  for item_index in range(0,len(x_list)):
    current = x_list[item_index]
    if current >999:
      if item_index == 0 or x_list[item_index -1 ] <999: 
        x_list[item_index] = counter
        counter = counter+1
      elif x_list[item_index -1 ] > 999:
        x_list[item_index] = x_list[item_index-1]
  
  

  if x_list[0] == 1000 or ((1000 in x_list) and sum_zero_before_1000(x_list,1000) == 0 ): 
    is_len_minus1 = 1
    for i in range(0,last_occurance_1000(x_list, 1000)+1):
      x_list[i] = 0

  
  return x_list , is_len_minus1




def stat_unemp(i, x_list,length):
  counter = 0

  if i>length-1 and i < len(x_list)-(2*length)+1 :    

    #Creating pre, current, post trackers for the interval 
    pre = [y for y in range(i-length, i)]
    trackers_pre = [x_list[y] for y in pre]
    current = [y for y in range(i, i+length)]
    trackers_current = [x_list[y] for y in current]
    post = [y for y in range(i+length, i+2*length)]
    trackers_post = [x_list[y] for y in post]


    if i == length:
      counter = 0 
      initial_iteration = trackers_pre + trackers_current

      j = len(initial_iteration)-1
      initial_current = initial_iteration[j-1]
      initial_after = initial_iteration[j]
      while j >=0:
        if initial_current > 999 and initial_after <999:  #end of spell   
         if j<length or initial_iteration[j-length] != initial_current: #spell is short enough  
            counter = counter +1
        initial_after = initial_current
        if j!=0: initial_current = initial_iteration[j-1]
        j -= 1
      
    else:
      counter = 0 

    only_this = only_this_interval(trackers_pre, trackers_current , trackers_post)
    unemp_trackers_current = [y for y in list(set(only_this)) if y > 999]


    if unemp_trackers_current != []:
      for tracker in unemp_trackers_current:
        indicator = (trackers_current[length-1] == tracker)
        counter = counter + indicator
       
  output = counter
  return output  


def thou_zero(y):
  if int(y) == 1000 or int(y) == 1003 : #or int(y) == 1007 :#or int(y) == 1005:
    return 0
  else: 
    return y



def unemployment_stats(x,tracker_list, tracker_list_job1):
  
  x_list = x[tracker_list_job1].tolist()
  x_list = [thou_zero(y) for y in x_list]
  x_list , is_len_minus1 = impute_unemp_tracker(x_list)
  
  unemp_stat1 = 0
  unemp_stat2 = 0
  unemp_stat3 = 0
  unemp_stat3_1 = 0
  unemp_stat8_drop = 0 
  total_unemp_spell = 0

  # print(x["ID"])
  # print(x_list)

  ##Find total unemployment spells , x =1 if x.index + 1 >999
  total_spells = 0
  for i in range(0,len(x_list)-1):
    if x_list[i] > 999 and x_list[i+1]<999 and x_list[i+1]>0  : total_spells = total_spells +1
    
    # Fraction of unemployment spells lasting less than 1 quarter
    if i > 0 and x_list[i]>999 and (x_list[i+1]<999 and x_list[i+1]>0) and (x_list[i-1]<999 and x_list[i-1]>0):
        unemp_stat1 = unemp_stat1 + 1

    # Fraction of unemployment spells lasting between 1 an 4 quarters
    unemp_stat2 = unemp_stat2 + stat_unemp(i,x_list,4)
      
    # Fraction of unemployment spells lasting between 4 to 8 quarters
    unemp_stat3 = unemp_stat3  + stat_unemp(i,x_list,8)

    # Fraction of unemployment spells lasting between 8 to 16 quarters
    unemp_stat3_1 = unemp_stat3_1  + stat_unemp(i,x_list,16)

    # Fraction of unemployment spells lasting between 8 to 16 quarters
    unemp_stat8_drop = unemp_stat8_drop  + stat_unemp(i,x_list,64)

  if is_len_minus1 ==1 :
    total_unemp_spell = max(x_list)-1000 
  else:
    total_unemp_spell = max(x_list)-999   
  if total_unemp_spell <0:   total_unemp_spell = 0
  #if total_unemp_spell <1 : print(str(x["ID"])+" "+str(total_unemp_spell)+ ",  Max_list = "+ str(max(x_list)) +", ump less 1 ")
  if total_unemp_spell > 0 :
    if unemp_stat1 <=unemp_stat2 and unemp_stat2<=unemp_stat3 and unemp_stat3<=unemp_stat3_1:
      if unemp_stat2 == 0 and unemp_stat3 == 0 and unemp_stat3 ==0 and unemp_stat3_1 ==0: #NO UNEMPLOYMENT SPELLS 
        x["stat5"] = 1
        x["stat7_2"] = 0
      else:
        x["stat5"] = (unemp_stat1 / total_unemp_spell)
        x["stat7_2"] = (total_unemp_spell-unemp_stat3_1)/total_unemp_spell

      x["stat6"] = (unemp_stat2 - unemp_stat1)/ total_unemp_spell
      x["stat7"] = (unemp_stat3 - unemp_stat2)/total_unemp_spell
      x["stat7_1"] = (unemp_stat3_1 - unemp_stat3)/total_unemp_spell
      x["stat8y_drop"] = (unemp_stat8_drop - unemp_stat3_1) / total_unemp_spell
    else: 
      print(str(x["ID"])+" "+str(total_unemp_spell)+ " " + str(unemp_stat1) + " "+ str(unemp_stat2) + " "+  str(unemp_stat3))
  elif total_unemp_spell == 0:
    x["stat5"] = 1
    x["stat6"] = 0
    x["stat7"] = 0
    x["stat7_1"] = 0
    x["stat7_2"] = 0
    x["stat8y_drop"] = 0


  sum_stat_8 = sum(is_unemployed(y) for y in x_list)
  x["stat8"] = sum_stat_8 / len(x_list)    
  return x 

def is_unemployed(y):
  output = 0 
  if y == 0 or y > 100:output = 1 
  return output




In [10]:
def summary_stat(x, id_list, tracker_list ):
  x  = x[id_list + tracker_list]
  x = employment_stats(x, tracker_list)

  tracker_list_job1 = [y for y in tracker_list if y[15]=="1"]
  x = unemployment_stats(x,tracker_list, tracker_list_job1)
      
  return x

##Loading and Cleaning

In [11]:

emp_hist_final1 = emp_hist_final #.sample(n=100)
id_list = ["ID", 	"SAMPLE_ID", 	"SAMPLE_RACE", 	"SAMPLE_SEX"]
job_tracker_list = [x for x in emp_hist_final1.columns.tolist() if x[:11] == "job_tracker"] 

emp_hist_stat = emp_hist_final1.apply(summary_stat, args= [id_list, job_tracker_list], axis =1)



29.0
62.0 8 3 6 8
96.0 10 4 9 8
97.0 7 2 5 6
107.0
119.0
160.0
224.0 12 10 11 11
237.0
250.0 4 0 1 1
255.0 16 1 11 14
267.0
290.0
292.0 11 4 9 9
297.0
309.0
341.0 2 0 1 2
346.0 5.0 1 4 4
349.0 5 3 3 4
356.0 13 5 11 13
363.0 13 3 7 10
365.0 9 2 6 6
385.0 7 1 5 5
391.0
391.0 15 8 12 12
399.0 16 9 15 15
405.0 5 0 3 5
434.0 5 2 2 4
469.0
474.0
531.0 10 2 3 5
555.0 12 7 11 12
612.0
622.0
667.0
678.0
721.0
724.0 17 10 14 14
800.0 4 3 4 3
812.0
833.0 2 0 1 2
855.0
886.0 12 3 9 12
893.0
903.0
917.0 13 1 9 12
925.0 7 0 5 7
936.0 9 3 8 9
951.0
956.0
972.0 8 5 7 7
977.0
978.0 16 11 16 16
1024.0
1041.0 11 2 3 4
1053.0
1061.0 5 3 5 4
1087.0
1088.0
1096.0 4 0 3 4
1106.0 26 14 24 28
1122.0 7 2 5 7
1169.0 6 1 5 4
1182.0 8 1 7 8
1193.0
1221.0
1259.0
1272.0 2 0 1 0
1339.0 15 3 9 12
1394.0
1433.0
1502.0
1529.0
1533.0
1597.0
1599.0
1622.0
1637.0
1649.0
1666.0
1703.0 20 10 20 20
1725.0
1733.0
1739.0 14 5 8 14
1770.0
1795.0 2 0 1 2
1803.0 11 4 9 11
1808.0 1 0 1 1
1820.0
1823.0
1842.0
1847.0 5 2 4 4
1858.0 5

In [12]:
new_stats = ["stat1", 	"stat2" ,	"stat3", "stat4","stat5", 	"stat6" ,	"stat7", "stat7_1", "stat7_2","stat8" 	,"stat9" , "stat8y_drop"]
emp_hist_stat[new_stats].describe()

,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat7_1,stat7_2,stat8,stat9,stat8y_drop
count,4202.000000,4202.000000,4202.000000,4202.000000,4216.000000,4216.000000,4216.000000,4216.000000,4216.000000,4333.000000,4333.000000,4216.000000
mean,0.150490,0.299031,0.146505,0.403974,0.292973,0.293792,0.131006,0.121105,0.161124,0.276072,0.077590,0.048873
std,0.133340,0.168209,0.115781,0.207814,0.250593,0.211548,0.144233,0.148339,0.181934,0.223659,0.042165,0.166853
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.333333,0.007968,0.000000,-1.000000
25%,0.000000,0.187500,0.062500,0.250000,0.125000,0.142857,0.000000,0.000000,0.000000,0.107570,0.047904,0.000000
50%,0.142857,0.307692,0.142857,0.375000,0.250000,0.285714,0.111111,0.090909,0.117647,0.199203,0.071856,0.000000
75%,0.230769,0.411765,0.222222,0.500000,0.400000,0.416667,0.200000,0.200000,0.266667,0.390438,0.101796,0.125000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.875000,1.000000,0.305389,1.000000


In [13]:
emp_hist_stat = emp_hist_stat.dropna()
emp_hist_stat = emp_hist_stat.reset_index()
emp_hist_stat[new_stats].describe()

,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat7_1,stat7_2,stat8,stat9,stat8y_drop
count,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000,4087.000000
mean,0.150754,0.299638,0.146485,0.403124,0.291375,0.295827,0.131224,0.120201,0.161373,0.275321,0.078274,0.049377
std,0.133417,0.168328,0.115726,0.208011,0.248703,0.211952,0.144436,0.147715,0.182274,0.220761,0.042015,0.167402
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.333333,0.007968,0.005988,-1.000000
25%,0.000000,0.187500,0.062500,0.250000,0.125000,0.142857,0.000000,0.000000,0.000000,0.107570,0.047904,0.000000
50%,0.142857,0.307692,0.142857,0.375000,0.250000,0.285714,0.111111,0.090909,0.117647,0.199203,0.071856,0.000000
75%,0.230769,0.413793,0.222222,0.500000,0.400000,0.421053,0.200000,0.200000,0.266667,0.390438,0.101796,0.125000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.875000,0.998008,0.275449,1.000000


In [14]:

#below are the ids that showed issues above
#there are 500 such ids which is 13.5 % of the data
#second iteration: there are 139 such ids which is 3 % of the data
id_to_drop = [29.0, 107.0, 119.0, 120.0, 160.0, 237.0, 267.0, 290.0, 297.0, 309.0, 346.0, 391.0, 469.0, 474.0, 589.0, 612.0, 622.0, 667.0, 678.0, 721.0, 812.0, 855.0, 880.0, 893.0, 903.0, 903.0, 951.0, 956.0, 977.0, 1024.0, 1053.0, 1087.0, 1088.0, 1147.0, 1193.0, 1221.0, 1259.0, 1394.0, 1433.0, 1502.0, 1529.0, 1533.0, 1597.0, 1599.0, 1622.0, 1637.0, 1649.0, 1666.0, 1725.0, 1733.0, 1770.0, 1820.0, 1823.0, 1842.0, 1975.0, 2077.0, 2093.0, 2113.0, 2134.0, 2164.0, 2229.0, 2247.0, 2283.0, 2347.0, 2381.0, 2415.0, 2462.0, 2482.0, 2512.0, 2551.0, 2569.0, 2572.0, 2654.0, 2658.0, 2732.0, 2755.0, 2851.0, 2868.0, 2874.0, 2885.0, 2920.0, 3088.0, 3195.0, 3275.0, 3280.0, 3287.0, 3359.0, 3416.0, 3428.0, 3473.0, 3486.0, 3546.0, 3570.0, 3685.0, 3700.0, 3809.0, 3888.0, 3902.0, 4070.0, 4108.0, 4265.0, 4326.0, 4370.0, 4474.0, 4498.0, 4537.0, 4580.0, 4656.0, 4673.0, 4687.0, 4748.0, 4838.0, 4840.0, 4855.0, 4872.0, 4892.0, 5050.0, 5060.0, 5197.0, 5219.0, 5260.0, 5337.0, 5387.0, 5395.0, 5459.0, 5465.0, 5479.0, 5514.0, 5540.0, 5554.0, 5653.0, 5697.0, 5712.0, 11779.0, 11805.0, 11961.0, 12054.0, 12070.0, 12116.0]
for id_d in id_to_drop:
  indexNames = emp_hist_stat[ emp_hist_stat["ID"] == id_d].index
  emp_hist_stat.drop(indexNames , inplace=True)


emp_hist_stat = emp_hist_stat.reset_index()
emp_hist_stat[new_stats].describe()



,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat7_1,stat7_2,stat8,stat9,stat8y_drop
count,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000
mean,0.150698,0.299522,0.146427,0.403353,0.290885,0.295557,0.131463,0.120420,0.161674,0.275564,0.078141,0.049439
std,0.133348,0.168436,0.115786,0.208154,0.248276,0.211722,0.144536,0.147841,0.182362,0.220927,0.041920,0.167582
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.333333,0.007968,0.005988,-1.000000
25%,0.000000,0.187500,0.060606,0.250000,0.125000,0.142857,0.000000,0.000000,0.000000,0.107570,0.047904,0.000000
50%,0.142857,0.307692,0.142857,0.375000,0.250000,0.285714,0.111111,0.090909,0.125000,0.199203,0.071856,0.000000
75%,0.230769,0.413793,0.222222,0.500000,0.400000,0.418860,0.200000,0.200000,0.266667,0.390438,0.101796,0.125000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.875000,0.998008,0.275449,1.000000


In [15]:
# # #Saving 
# emp_hist_stat_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Analysis/Temp_df/emp_hist_stat.csv'
# with open(emp_hist_stat_path, 'w', encoding = 'utf-8-sig') as f:
#   emp_hist_stat.to_csv(f)

In [16]:
# emp_hist_stat_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Analysis/Temp_df/emp_hist_stat.csv'
# emp_hist_stat = pd.read_csv(emp_hist_stat_path)
# emp_hist_stat.drop('Unnamed: 0', inplace=True, axis=1)

In [17]:
emp_hist_stat.drop('level_0', inplace=True, axis=1)
emp_hist_stat.drop('index', inplace=True, axis=1)

new_stats = ["stat1", 	"stat2" ,	"stat3", "stat4","stat5", 	"stat6" ,	"stat7", "stat7_1","stat7_2", "stat8" 	,"stat9"  ]
id_list = ["ID", 	"SAMPLE_ID", 	"SAMPLE_RACE", 	"SAMPLE_SEX"]



emp_hist_stat[new_stats].describe()

,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat7_1,stat7_2,stat8,stat9
count,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000,4075.000000
mean,0.150698,0.299522,0.146427,0.403353,0.290885,0.295557,0.131463,0.120420,0.161674,0.275564,0.078141
std,0.133348,0.168436,0.115786,0.208154,0.248276,0.211722,0.144536,0.147841,0.182362,0.220927,0.041920
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.333333,0.007968,0.005988
25%,0.000000,0.187500,0.060606,0.250000,0.125000,0.142857,0.000000,0.000000,0.000000,0.107570,0.047904
50%,0.142857,0.307692,0.142857,0.375000,0.250000,0.285714,0.111111,0.090909,0.125000,0.199203,0.071856
75%,0.230769,0.413793,0.222222,0.500000,0.400000,0.418860,0.200000,0.200000,0.266667,0.390438,0.101796
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.875000,0.998008,0.275449


In [18]:
emp_hist_stat[emp_hist_stat["stat7_2"] == 0 ][new_stats].describe()

emp_hist_stat = emp_hist_stat[emp_hist_stat["stat7_2"] == 0 ]

In [19]:
new_stats = ["stat1", 	"stat2" ,	"stat3", "stat4","stat5", 	"stat6" ,	"stat7", "stat7_1" ,"stat7_2" , "stat8" 	,"stat9"  ]
id_list = ["ID", 	"SAMPLE_ID", 	"SAMPLE_RACE", 	"SAMPLE_SEX"]

emp_hist_stat = emp_hist_stat.reset_index()
if 'level_0' in list(emp_hist_stat.columns): emp_hist_stat.drop('level_0', inplace=True, axis=1)
if 'index' in list(emp_hist_stat.columns): emp_hist_stat.drop('index', inplace=True, axis=1)
emp_hist_stat

,ID,SAMPLE_ID,SAMPLE_RACE,SAMPLE_SEX,job_tracker_job1-0003,job_tracker_job1-0007,job_tracker_job1-0011,job_tracker_job1-0015,job_tracker_job1-0020,job_tracker_job1-0024,...,stat3,stat4,stat5,stat6,stat7,stat7_1,stat7_2,stat8,stat8y_drop,stat9
0,2.0,5.0,3.0,2.0,1005.0,1005.0,1004.0,1005.0,1.0,1.0,...,0.250000,0.750000,1.000000,0.000000,0.000000,0.000000,0.0,0.286853,0.000000,0.023952
1,6.0,1.0,3.0,1.0,1005.0,1005.0,1005.0,1005.0,1.0,1.0,...,0.000000,0.454545,0.333333,0.444444,0.222222,0.000000,0.0,0.089641,0.000000,0.065868
2,8.0,6.0,3.0,2.0,1005.0,1005.0,1005.0,1005.0,1005.0,1005.0,...,0.000000,0.384615,0.200000,0.400000,0.200000,0.200000,0.0,0.095618,0.000000,0.077844
3,9.0,1.0,3.0,1.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,0.000000,0.600000,0.000000,0.333333,0.000000,0.666667,0.0,0.133466,0.000000,0.029940
4,14.0,5.0,3.0,2.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,0.230769,0.230769,0.250000,0.312500,0.312500,0.125000,0.0,0.217131,-0.125000,0.155689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,12124.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.250000,0.625000,1.000000,0.000000,0.000000,0.000000,0.0,0.256972,0.333333,0.047904
1506,12125.0,3.0,2.0,1.0,1005.0,1005.0,1005.0,1005.0,1005.0,1005.0,...,0.181818,0.272727,0.250000,0.375000,0.375000,0.000000,0.0,0.143426,0.000000,0.065868
1507,12128.0,5.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.222222,0.444444,0.000000,1.000000,0.000000,0.000000,0.0,0.027888,0.000000,0.053892
1508,12131.0,3.0,2.0,1.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.139442,1.000000,0.023952


In [20]:
emp_hist_stat[new_stats].describe()

,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat7_1,stat7_2,stat8,stat9
count,1510.000000,1510.000000,1510.000000,1510.000000,1510.000000,1510.000000,1510.000000,1510.000000,1510.0,1510.000000,1510.000000
mean,0.121468,0.254917,0.134670,0.488946,0.410309,0.335999,0.135801,0.117891,0.0,0.129506,0.068426
std,0.136044,0.170897,0.116423,0.215732,0.314668,0.263646,0.179198,0.181817,0.0,0.150854,0.040308
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.007968,0.005988
25%,0.000000,0.142857,0.000000,0.333333,0.200000,0.142857,0.000000,0.000000,0.0,0.059761,0.035928
50%,0.100000,0.250000,0.133333,0.466667,0.363636,0.333333,0.076923,0.000000,0.0,0.093625,0.059880
75%,0.187500,0.368421,0.210526,0.615385,0.571429,0.500000,0.230769,0.200000,0.0,0.143426,0.089820
max,1.000000,1.000000,0.750000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.998008,0.251497


In [21]:
# # #Saving 
# emp_hist_stat_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Analysis/Temp_df/emp_hist_stat.csv'
# with open(emp_hist_stat_path, 'w', encoding = 'utf-8-sig') as f:
#   emp_hist_stat.to_csv(f)


# emp_hist_stat_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Analysis/Temp_df/emp_hist_stat.csv'
# emp_hist_stat = pd.read_csv(emp_hist_stat_path)
# emp_hist_stat.drop('Unnamed: 0', inplace=True, axis=1)

#K-means with three clusters

In [22]:
emp_hist_stat_path = '/content/drive/MyDrive/Alpha_beta_gamma/Coding/Analysis/Temp_df/emp_hist_stat.csv'
emp_hist_stat = pd.read_csv(emp_hist_stat_path)
emp_hist_stat.drop('Unnamed: 0', inplace=True, axis=1)
emp_hist_stat

,ID,SAMPLE_ID,SAMPLE_RACE,SAMPLE_SEX,job_tracker_job1-0003,job_tracker_job1-0007,job_tracker_job1-0011,job_tracker_job1-0015,job_tracker_job1-0020,job_tracker_job1-0024,...,stat3,stat4,stat5,stat6,stat7,stat7_1,stat7_2,stat8,stat8y_drop,stat9
0,2.0,5.0,3.0,2.0,1005.0,1005.0,1004.0,1005.0,1.0,1.0,...,0.250000,0.750000,1.000000,0.000000,0.000000,0.000000,0.0,0.286853,0.000000,0.023952
1,6.0,1.0,3.0,1.0,1005.0,1005.0,1005.0,1005.0,1.0,1.0,...,0.000000,0.454545,0.333333,0.444444,0.222222,0.000000,0.0,0.089641,0.000000,0.065868
2,8.0,6.0,3.0,2.0,1005.0,1005.0,1005.0,1005.0,1005.0,1005.0,...,0.000000,0.384615,0.200000,0.400000,0.200000,0.200000,0.0,0.095618,0.000000,0.077844
3,9.0,1.0,3.0,1.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,0.000000,0.600000,0.000000,0.333333,0.000000,0.666667,0.0,0.133466,0.000000,0.029940
4,14.0,5.0,3.0,2.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,0.230769,0.230769,0.250000,0.312500,0.312500,0.125000,0.0,0.217131,-0.125000,0.155689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,12124.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.250000,0.625000,1.000000,0.000000,0.000000,0.000000,0.0,0.256972,0.333333,0.047904
1506,12125.0,3.0,2.0,1.0,1005.0,1005.0,1005.0,1005.0,1005.0,1005.0,...,0.181818,0.272727,0.250000,0.375000,0.375000,0.000000,0.0,0.143426,0.000000,0.065868
1507,12128.0,5.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.222222,0.444444,0.000000,1.000000,0.000000,0.000000,0.0,0.027888,0.000000,0.053892
1508,12131.0,3.0,2.0,1.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.139442,1.000000,0.023952


In [23]:
new_stats = ["stat1", 	"stat2" ,	"stat3", "stat4","stat5", 	"stat6" ,	"stat7", "stat7_1" ,"stat7_2" , "stat8" 	,"stat9"  ]
id_list = ["ID", 	"SAMPLE_ID", 	"SAMPLE_RACE", 	"SAMPLE_SEX"]

In [24]:
# Convert DataFrame to matrix
mat = emp_hist_stat[new_stats].values

# Using sklearn
km = kmean(n_clusters=3, n_init = 100).fit(mat)

# # Get cluster assignment labels
labels = km.labels_

# Format results as a DataFrame
results = pd.DataFrame([emp_hist_stat[new_stats].index,labels]).T
results.columns = ["Index", "Kmeans_3"]
results = results.drop(columns=['Index'])


emp_hist_stat_kmeans = pd.concat([emp_hist_stat, results], axis="columns")


value_count_kmeans_3 = emp_hist_stat_kmeans["Kmeans_3"].value_counts()/len(emp_hist_stat_kmeans.index) 
print(value_count_kmeans_3)
print(emp_hist_stat_kmeans[new_stats + ["Kmeans_3"]].groupby("Kmeans_3").mean())



1    0.492053
2    0.264901
0    0.243046
Name: Kmeans_3, dtype: float64
             stat1     stat2     stat3     stat4     stat5     stat6  \
Kmeans_3                                                               
0         0.106595  0.212111  0.109387  0.571908  0.866273  0.078917   
1         0.142606  0.298777  0.148806  0.409811  0.293924  0.286759   
2         0.095848  0.212721  0.131611  0.559821  0.208150  0.663334   

             stat7   stat7_1  stat7_2     stat8     stat9  
Kmeans_3                                                   
0         0.029967  0.024844      0.0  0.185416  0.046060  
1         0.230400  0.188917      0.0  0.125249  0.083792  
2         0.057186  0.071331      0.0  0.086116  0.060404  


In [25]:

col_list_1 = ["ID","SAMPLE_ID", "SAMPLE_RACE", 	"SAMPLE_SEX", "Kmeans_3"]
col_list_2 = [x for x in emp_hist_stat_kmeans.columns.tolist() if x[:11] =="job_tracker" ]
col_list_2 = sorted(col_list_2, key=lambda x: x[-4:])

df_input = emp_hist_stat_kmeans[col_list_1 + col_list_2]

In [26]:
df_input

,ID,SAMPLE_ID,SAMPLE_RACE,SAMPLE_SEX,Kmeans_3,job_tracker_job1-0003,job_tracker_job2-0003,job_tracker_job3-0003,job_tracker_job4-0003,job_tracker_job1-0007,...,job_tracker_job3-2174,job_tracker_job4-2174,job_tracker_job1-2178,job_tracker_job2-2178,job_tracker_job3-2178,job_tracker_job4-2178,job_tracker_job1-2182,job_tracker_job2-2182,job_tracker_job3-2182,job_tracker_job4-2182
0,2.0,5.0,3.0,2.0,0,1005.0,1005.0,1005.0,1005.0,1005.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
1,6.0,1.0,3.0,1.0,1,1005.0,1005.0,1005.0,1005.0,1005.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
2,8.0,6.0,3.0,2.0,1,1005.0,1005.0,1005.0,1005.0,1005.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
3,9.0,1.0,3.0,1.0,1,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
4,14.0,5.0,3.0,2.0,1,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,12124.0,3.0,2.0,1.0,0,1.0,0.0,0.0,0.0,1.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
1506,12125.0,3.0,2.0,1.0,1,1005.0,1005.0,1005.0,1005.0,1005.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
1507,12128.0,5.0,3.0,2.0,2,1.0,0.0,0.0,0.0,1.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
1508,12131.0,3.0,2.0,1.0,0,1000.0,1000.0,1000.0,1000.0,1000.0,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0


##The Matching 

In [27]:
y_axis = [[] for x in range(1,31)]


In [48]:
def find_tenure(x):
  # x = x.tolist()
 # print(x)
  # print(len(x))

  #print(list(set(x[-8:-4])))
  current_jobs = np.array([y for y in list(set(x[-8:-4])) if y <= 100 and y > 0 ])
  tenure = 0 
  #print(current_jobs)
  for i in current_jobs:
    tenure_i = np.count_nonzero(x[:-4]  == i)
    if tenure_i > tenure : tenure = tenure_i
  #print(tenure)
  output = tenure
  return output 


tenure_list = []
i = 31 
while i < 32 :#len(col_list_2)/4  :
  a = col_list_2[(i * 4) - 31 * 4  : (i * 4 )  ]

  values_i = df_input[col_list_2[(i * 4) - 31 * 4  : (i * 4 )  ]].values
  # print(df_input[col_list_2[(i * 4) - 31 * 4  : (i * 4 )  ]].columns.tolist())
  print(values_i[3].tolist())
  # print(find_tenure([values_i[0]]))
  tenure = [find_tenure(x) for x in values_i]
  tenure_list.append(tenure)
  print(tenure[3])

  # print(" *********************** ")


  i = i +1




[1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1000.0, 1003.0, 1003.0, 1003.0, 1003.0, 1.0, 0.0, 0.0, 0.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1004.0, 1004.0, 1004.0, 1004.0, 1004.0, 1004.0, 1004.0, 1004.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 1005.0, 3.0, 2.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 3.0, 2.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0]
6


In [29]:
print(len(tenure_list[8]))
tenure_by_person = []

for year in tenure_list[1]:
  tenure_by_person.append([])

print(len(tenure_by_person))

for year in tenure_list:
  i = 0 
  for person_tenure in year:
    tenure_by_person[i].append(person_tenure)
    i = i+1




1510
1510


In [30]:

all_signal = []
for i in tenure_by_person:
  j = 1 
  signal = 0 
  while j < len(i):
    if i[j] > 8 and i[j-1] == 0: 
      signal = 1 
      break 
    j = j + 1

  all_signal.append(signal)

print(mean(all_signal))
    
    

# print("******************")
# print("******************")
# print("******************")

# for i in tenure_list[:30]:
#   print(i)

0.6576158940397351


In [31]:


a = [1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
b = [1.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0]

print(len([x for x in a if x ==1.0]))
print(len([x for x in b if x ==1.0]))


26
26


In [32]:
print(col_list_2)

['job_tracker_job1-0003', 'job_tracker_job2-0003', 'job_tracker_job3-0003', 'job_tracker_job4-0003', 'job_tracker_job1-0007', 'job_tracker_job2-0007', 'job_tracker_job3-0007', 'job_tracker_job4-0007', 'job_tracker_job1-0011', 'job_tracker_job2-0011', 'job_tracker_job3-0011', 'job_tracker_job4-0011', 'job_tracker_job1-0015', 'job_tracker_job2-0015', 'job_tracker_job3-0015', 'job_tracker_job4-0015', 'job_tracker_job1-0020', 'job_tracker_job2-0020', 'job_tracker_job3-0020', 'job_tracker_job4-0020', 'job_tracker_job1-0024', 'job_tracker_job2-0024', 'job_tracker_job3-0024', 'job_tracker_job4-0024', 'job_tracker_job1-0028', 'job_tracker_job2-0028', 'job_tracker_job3-0028', 'job_tracker_job4-0028', 'job_tracker_job1-0033', 'job_tracker_job2-0033', 'job_tracker_job3-0033', 'job_tracker_job4-0033', 'job_tracker_job1-0037', 'job_tracker_job2-0037', 'job_tracker_job3-0037', 'job_tracker_job4-0037', 'job_tracker_job1-0041', 'job_tracker_job2-0041', 'job_tracker_job3-0041', 'job_tracker_job4-0041',

In [33]:
col_list_2[-8:-4]

['job_tracker_job1-2178',
 'job_tracker_job2-2178',
 'job_tracker_job3-2178',
 'job_tracker_job4-2178']